 # YOLO Container Classificatie
 Dit notebook traint een YOLOv8/YOLOv9 classificatiemodel om te bepalen:
 **"Staat er een container op de foto?"**

 We gebruiken hiervoor de dataset die al bestaat:

 temp_data/
   train/
     container/
     geen_container/
   val/
     container/
     geen_container/


In [1]:
from ultralytics import YOLO
import os

In [9]:
import os

print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/train/container"))
print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/train/geen_container"))
print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val/container"))
print(os.path.exists("C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val/geen_container"))


True
True
True
True


In [12]:
import yaml

with open("yolo_data.yaml", "r") as f:
    data = yaml.safe_load(f)

print(data)


{'train': 'C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/train', 'val': 'C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data/val', 'nc': 2, 'names': ['container', 'geen_container']}


In [ ]:
val_results = model.val()
print(val_results.metrics)  # bevat accuracy, precision, recall


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true, y_pred = [], []

for img_path in sample_imgs:
    pred = model.predict(img_path)[0]
    label_pred = pred.labels[0] if len(pred.labels) > 0 else -1
    y_pred.append(label_pred)
    y_true.append(0 if 'container' in img_path else 1)

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))


In [ ]:
from ultralytics import YOLO

# Laad YOLOv8-classification model
model = YOLO("yolov8n-cls.pt")

# Geef hier **alleen de root van de dataset**, niet het YAML
data_root = r"C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data"

results = model.train(
    data=data_root,  # <--- hier de map, niet YAML
    epochs=4,
    batch=32,
    imgsz=224,
    lr0=0.001,  # let op: gebruik lr0, niet lr
    name="container_classification"
)

Ultralytics 8.3.229  Python-3.11.9 torch-2.9.1+cpu CPU (12th Gen Intel Core i7-12650H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/rtvan/Documents/GitHub/AI-Waste_Vision_POC/Container Detection/MobileNet/temp_data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=4, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=container_classification3, nbs=64, nms=False, opset=None, optimize=F

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

sample_imgs = [
    os.path.join(data_root, 'val', 'container', os.listdir(os.path.join(data_root, 'val', 'container'))[0]),
    os.path.join(data_root, 'val', 'geen_container', os.listdir(os.path.join(data_root, 'val', 'geen_container'))[0])
]

plt.figure(figsize=(8,4))
for i, img_path in enumerate(sample_imgs):
    img = Image.open(img_path)
    plt.subplot(1,2,i+1)
    plt.imshow(img)
    plt.title(os.path.basename(os.path.dirname(img_path)))
    plt.axis('off')
plt.show()
